<h2>2022-05-17: USE WITH 'fastsim_upd' BRANCH</h2>

In [ ]:
#del sys.path[...]
#sys.path.insert(0,'.../epix') # 'fastsim_upd' branch

In [ ]:
import warnings
warnings.filterwarnings("ignore", message="You are setting a default value")

import time
import sys

import numba
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, FormatStrFormatter, AutoMinorLocator
from collections import Counter
import matplotlib.ticker as ticker

import epix
import straxen
straxen.print_versions()

from epix.simulator.fast_simulator import StraxSimulator

<h2>Convenience functions</h2>

In [ ]:
def ax_config(ax, title = None, xbounds = None, ybounds = None, xlabel = None, ylabel = None, 
              grid_M_X = True, grid_M_Y = True, grid_m_X = True, grid_m_Y = True,
              colorbar = False, clabel='', sci = False, yscale = None, xscale = None, fontsize=20):
    Label_fontname = 'serif'
    if title != None: ax.set_title(title, fontname=Label_fontname)
    if xbounds != None: ax.set_xlim(xbounds[0], xbounds[1])
    if ybounds != None: ax.set_ylim(ybounds[0], ybounds[1])
    if xlabel != None: ax.set_xlabel(xlabel, fontname=Label_fontname)
    if ylabel != None: ax.set_ylabel(ylabel, fontname=Label_fontname)
    if xscale != None: ax.set_xscale(xscale)
    if yscale != None: ax.set_yscale(yscale)
        
    if isinstance(sci, str):
        plt.ticklabel_format(style='sci', axis=sci, scilimits=(0,0))
        
    if xscale == 'log':
        locmin = matplotlib.ticker.LogLocator(base=10.0,subs=(0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9),numticks=12)
        ax.xaxis.set_minor_locator(locmin)
        ax.xaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())
    else:
        ax.xaxis.set_minor_locator(AutoMinorLocator(10))
        
    if yscale == 'log':
        locmin = matplotlib.ticker.LogLocator(base=10.0,subs=(0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9),numticks=12)
        ax.yaxis.set_minor_locator(locmin)
        ax.yaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())
    else:
        ax.yaxis.set_minor_locator(AutoMinorLocator(10))
    
    if grid_M_X:
        ax.xaxis.grid(grid_M_X, which='major', color='black', alpha=0.1)
    if grid_M_Y:
        ax.yaxis.grid(grid_M_Y, which='major', color='black', alpha=0.1)
    if grid_m_X:
        ax.xaxis.grid(grid_m_X, which='minor', color='gray', alpha=0.1)
    if grid_m_Y:
        ax.yaxis.grid(grid_m_Y, which='minor', color='gray', alpha=0.1)
    
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(fontsize)
        
    if isinstance(colorbar,str):
        plt.colorbar().set_label(label = colorbar,size=12, weight = None)
    elif colorbar:
        plt.colorbar(label = clabel)
    else:
        pass
    
    ax.set_facecolor('white')

<h2>Path</h2>

In [ ]:
# AmBe source, I-Belt position, tungsten cylinder collimator, 1E4 primaries
raw_data_dir = '/dali/lgrandi/xenonnt/simulations/ambe_ibelt/mc_v4.4.0/tungsten_cylinder_GEANT4/'
raw_data_filename='ibc_008f8efb-99ac-475a-a83a-9c35cca8734d.root'

# Your local directory for strax output
strax_data_dir = '/your/scratch/directory/strax'

In [ ]:
# Clear old strax output
%rm -r /your/scratch/directory/strax/*

<h2>Parameters</h2>

In [ ]:
st = straxen.contexts.xenonnt_simulation(output_folder=strax_data_dir,
                                         cmt_run_id_sim='026000',
                                         cmt_run_id_proc='026000',)
run_id = 'fastsim0'

# epix config
epix_args={'path':raw_data_dir,
           'file_name':raw_data_filename, 
           'debug':True,           
           'entry_start':0,
           'entry_stop':None,
           'cut_by_eventid':False,
           'micro_separation_time':10.0,
           'micro_separation':0.005,
           'tag_cluster_by':'energy',
           'max_delay':1e7,
           'source_rate':-1}

configuration_files={'nv_pmt_qe':'nveto_pmt_qe.json',
                     'photon_area_distribution':'XENONnT_spe_distributions_20210305.csv',
                     's1_pattern_map': 'XENONnT_s1_xyz_patterns_LCE_corrected_qes_MCva43fa9b_wires.pkl',
                     's2_pattern_map': 'XENONnT_s2_xy_patterns_GXe_LCE_corrected_qes_MCv4.3.0_wires.pkl',
                     's1_lce_correction_map': False,
                     's2_correction_map': False}

# List of disabled PMTs
_gains = [1 for i in range(494)]
pmt_off = [156, 164, 177, 350, 352, 354, 362, 386, 393, 427, 313]
for _pmt in pmt_off:
    _gains[_pmt]=0
    
# Set up configuration
st.config.update(dict(fax_config='fax_config_nt_sr0_v0.json',                      
                      g4_file=raw_data_dir+raw_data_filename,
                      epix_config=epix_args,
                      configuration_files=configuration_files,
                      fax_config_overrides={"detector":"XENONnT",
                                            "gains": _gains,
                                            "enable_field_dependencies": {                                                
                                                "survival_probability_map": False,
                                                "drift_speed_map": False,
                                                "diffusion_longitudinal_map": False,
                                                "diffusion_radial_map": False,
                                                "diffusion_azimuthal_map": False}
                                           }))

st.register(StraxSimulator)

<h2>Run simulation</h2>

In [ ]:
start_time = time.time()

st.make(run_id,'events_tpc')
st.make(run_id, 'events_nveto')

t = time.time()
print(f'It took {(t - start_time):.1f} [s] to finish the simulation.')

<h2>events_tpc</h2>

In [ ]:
_events_tpc = st.get_array(run_id, 'events_tpc')
df_events_tpc = pd.DataFrame(_events_tpc)
df_events_tpc

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=[22,10])

rr=[ [(0.1, 5.0), (0.1, 8.0)],
     [(0.1, 5.0), (0.1, 8.0)],    
     [(-80, 80), (-80, 80)],
     [(0.1, 5.0), (0.1, 8.0)],
     [(0.1, 5.0), (0.1, 8.0)],
     [(0.0, 7e3), (-150.0, 0.0)] ]

label_list = [ ['log(s1_area)', 'log(s2_area)'],
               ['log(alt_s1_area)', 'log(alt_s2_area)'],
               ['x [cm]','y [cm]'],
               ['log(cs1)', 'log(cs2)'],
               ['log(alt_cs1)', 'log(alt_cs2)'],
               ['r**2 [cm**2]', 'z [cm]'] ]

_ei = df_events_tpc

arr_list = [ [np.log10(_ei.s1_area), np.log10(_ei.s2_area)],
             [np.log10(_ei.alt_s1_area), np.log10(_ei.alt_s2_area)],
             [_ei.x, _ei.y],
             [np.log10(_ei.cs1), np.log10(_ei.cs2)],
             [np.log10(_ei.alt_cs1), np.log10(_ei.alt_cs2)], 
             [(_ei.x**2+_ei.y**2), _ei.z] ]

for ind, arr in enumerate(arr_list):
    _ax = ax[ind//3][ind%3]

    _ax.scatter(arr[0], 
                arr[1],
                color = 'blue',
                s=12,
                alpha=0.8)  

    ax_config(_ax, 
              xbounds = rr[ind][0],
              ybounds = rr[ind][1],
              xlabel=label_list[ind][0], 
              ylabel=label_list[ind][1])
    
    _ax.ticklabel_format(axis='x', style='sci', scilimits=(0,3))
    _ax.ticklabel_format(axis='y', style='sci', scilimits=(0,3))

plt.tight_layout()
plt.plot()

<h2>events_nveto</h2>

In [ ]:
df_events_nv = st.get_df(run_id, 'events_nveto')
df_events_nv